# Skin lesion classification

**Deadline**: Upload this notebook (rename it as 'TP1-DL-YOUR-SURNAME.ipynb') to Ecampus/Moodle before the deadline.


**Context**
A skin lesion is defined as a superficial growth or patch of the skin that is visually different and/or has a different texture than its surrounding area. Skin lesions, such as moles or birthmarks, can degenerate and become melanoma, one of the deadliest skin cancer. Its incidence has been increasing during the last decades, especially in the areas mostly populated by white people.

The most effective treatment is an early detection followed by surgical excision. This is why several approaches for melanoma detection have been proposed in the last years (non-invasive computer-aided diagnosis (CAD) ).

**Data**
You will have at your disposal the ISIC 2017 dataset (https://challenge.isic-archive.com/data/#2017) already pre-processed, resized and quality checked. It is divided into Training (N=2000), Validation (N=150) and Test (N=600) sets.

**Goal**
The goal of this practical session is to classify images of skin lesions as either benign (nevus or seborrheic_keratosis) or melanoma (binary classification) using machine and deep learning algorithms.

In the first part of the TP, you will manually compute some features relevant to the skin lesion classification (feature engineering) and then classify images using "classical" ML algorithms such as, logistic regression, SVM and Random Forests.

In the second part, you will test the features learnt with Deep Learning algorithms. You will first train from scratch well-known CNN architectures (VGG, ResNet, DenseNet, etc..) and then leverage the representations learnt by these networks on a pre-training from Imagenet (fine-tuning, full-restimation).

Please complete the code where you see **"XXXXXXXXX"** and answer the **Questions**


In [ ]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.io import imsave
from skimage.transform import resize
from skimage import color
from skimage import measure
from skimage import transform
from skimage.color import rgb2gray
from scipy import ndimage
from scipy import stats
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import torch
import glob
import cv2
from PIL import Image
from PIL import ImageShow

# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset, TensorDataset
from torchvision import models,transforms
!pip install torchmetrics
import torchmetrics

# torchvision
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import InterpolationMode


# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


%matplotlib inline

try:
  import google.colab
  IN_COLAB = True
  !pip install gdown==4.6.0 # with the following versions, there is an error
except:
  IN_COLAB = False

You can either download the data from my Google Drive or work locally.

In [ ]:
if IN_COLAB:
  print("you are using google colab")
  import gdown
  !mkdir ./data
  gdown.download(id="1iH5hkRN0wCgGklUN5df9u2Ue3UXAR4xZ", output='./data/TrainCropped.zip', quiet=False)
  !unzip -qu "./data/TrainCropped.zip" -d "./data"
  gdown.download(id="1lyRZuV9UST55AEqwSy4mqMmh5yHGI1FM", output='./data/TestCropped.zip', quiet=False)
  !unzip -qu "./data/TestCropped.zip" -d "./data"
  gdown.download(id="1RLJOmqAnHCgiJ7qShQurpxNaRhjjPpJb", output='./data/ValCropped.zip', quiet=False)
  !unzip -qu "./data/ValCropped.zip" -d "./data"
  !rm -rf ./data/TrainCropped.zip
  !rm -rf ./data/TestCropped.zip
  !rm -rf ./data/ValCropped.zip
  path='./data/'
else:
  print('You are NOT using colab')
  # we assume that folders of data are in the same folder as this jupyter notebook
  path='' # if you change this path , you should also change idTRain, idVal and idTest


If there is an error (might happen with gdown) please upload the three files manually.
Follow the following instructions:
- go to the folder symbol on the left of your screen
- click on the three vertical dots on the 'data' folder
- upload (importer in french) the three folders
That's it !

In [ ]:
# if IN_COLAB:
#   !unzip -qu "./data/TrainCropped.zip" -d "./data"
#   !unzip -qu "./data/TestCropped.zip" -d "./data"
#   !unzip -qu "./data/ValCropped.zip" -d "./data"
#   !rm -rf ./data/TrainCropped.zip
#   !rm -rf ./data/TestCropped.zip
#   !rm -rf ./data/ValCropped.zip
#   path='./data/'

For the Deep Learning part, we strongly suggest using GPU

In [ ]:
if torch.cuda.is_available():
  print('Is there a GPU card?', torch.cuda.is_available(),'\nNumber of GPU cards: ', torch.cuda.device_count(), '\nWhich card GPU?', torch.cuda.get_device_name(0))
  print('Total GPU memory {1:.2f} GB. Free GPU memory {0:.2f} GB'.format(torch.cuda.mem_get_info()[0]/pow(10,9),torch.cuda.mem_get_info()[1]/pow(10,9)))

Let's load the data.

In [ ]:
pathTrain=glob.glob(path + "TrainCropped/*.jpg")
print(pathTrain)
idTrain=np.copy(pathTrain)
if IN_COLAB:
    for i in np.arange(len(idTrain)): idTrain[i]=idTrain[i][20:-4]
else:
    for i in np.arange(len(idTrain)): idTrain[i]=idTrain[i][13:-4]
#print(idTrain)
print('There are', len(idTrain), 'Train images')

In [ ]:
pathVal=glob.glob(path + "ValCropped/*.jpg")
idVal=np.copy(pathVal)
if IN_COLAB:
    for i in np.arange(len(idVal)): idVal[i]=idVal[i][18:-4]
else:
    for i in np.arange(len(idVal)): idVal[i]=idVal[i][11:-4]
#print(idVal)
print('There are', len(idVal) , 'Validation images')

In [ ]:
pathTest=glob.glob(path + "TestCropped/*.jpg")
idTest=np.copy(pathTest)
if IN_COLAB:
    for i in np.arange(len(idTest)): idTest[i]=idTest[i][19:-4]
else:
    for i in np.arange(len(idTest)): idTest[i]=idTest[i][12:-4]
#print(idTest)
print('There are', len(idTest) , 'Test images')

## Loading Metadata and Target values

You have at your disposal also two metadata, the age and the sex. If you want, you can use them as features in the classification but be careful ! There are missing values

We also load the target values (0 for benign and 1 for melanoma)

In [ ]:
Metatrain = pd.read_csv('./data/TrainCropped/ISIC-2017_Training_Data_metadata.csv')
print(Metatrain.head(10))
Groundtrain = pd.read_csv('./data/TrainCropped/ISIC-2017_Training_Part3_GroundTruth.csv')

In [ ]:
Ytrain=np.zeros(len(idTrain))
for i in range(len(idTrain)):
  name=idTrain[i]
  index=Groundtrain["image_id"].str.find(name)
  max_index = index.argmax()
  Ytrain[i]=int(Groundtrain["melanoma"][max_index])

In [ ]:
Metaval = pd.read_csv('./data/ValCropped/ISIC-2017_Validation_Data_metadata.csv')
Groundval = pd.read_csv('./data/ValCropped/ISIC-2017_Validation_Part3_GroundTruth.csv')

In [ ]:
Yval=np.zeros(len(idVal))
for i in range(len(idVal)):
  name=idVal[i]
  index=Groundval["image_id"].str.find(name)
  max_index = index.argmax()
  Yval[i]=int(Groundval["melanoma"][max_index])

In [ ]:
Metatest = pd.read_csv('./data/TestCropped/ISIC-2017_Test_v2_Data_metadata.csv')
Groundtest = pd.read_csv('./data/TestCropped/ISIC-2017_Test_v2_Part3_GroundTruth.csv')

In [ ]:
Ytest=np.zeros(len(idTest))
for i in range(len(idTest)):
  name=idTest[i]
  index=Groundtest["image_id"].str.find(name)
  max_index = index.argmax()
  Ytest[i]=int(Groundtest["melanoma"][max_index])

##Deep Learning
In this section, you will try simple Deep Learning strategies:
- Fine-tuning the last layer of a network already trained on Image-Net
- Re-training completely a network already trained on Image-Net
- Re-training from scratch a network that has already shown good performances on other data-sets (architecture transfer or inductive bias transfer)

You will have to use at least three different networks (e.g., ResNet, VGG, DenseNet)

**Please compute the full time (reading papers/tutorials, coding, computational time) you spend on this part. It will be asked at the end of the practical session**

Pytorch also offers two nice primitives for storing and working with datasets:

*   **Dataset** stores the images, labels and segmentation masks
*   **DataLoader** wraps an iterable around the elements of the Dataset

This is very practical since we can easily resize the images to the same size (important for DL) and multiply image and segmentation masks. Here, you have an exemple.

In [ ]:
class ISICDataset(Dataset):
    def __init__(self, pathlist, targets, size=(224,224)):
        self.image_paths = pathlist
        self.mask_paths = [ p[:-4]+'seg.png' for p in pathlist ]
        self.targets = torch.LongTensor(targets)
        self.size=size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        y = self.targets[idx]

        x = Image.open(img_path).resize(self.size, Image.BILINEAR) # all images are resized to (224,224)
        s = Image.open(mask_path).resize(self.size, Image.NEAREST) # all segmentation masks are resized to (224,224)

        # Multiply image and segmentation mask
        blank = x.point(lambda _: 0)
        c = Image.composite(x, blank, s)

        # Send to tensor
        x = TF.to_tensor(x)
        c = TF.to_tensor(c)

        # Normalize according to ImageNet statistics
        x = TF.normalize(x,(0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        c = TF.normalize(c,(0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

        return x, c, y


Another reason why DataSets and DataLoaders are practical is that we can automatically apply data augmentation strategies. For instance, if we want to automatically apply (the same) data augmentations to both images and segmentations, we can use:

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


class ISICDatasetWithAug(Dataset):
    def __init__(self, pathlist, targets, size=(224,224)):
        self.image_paths = pathlist
        self.mask_paths = [ p[:-4]+'seg.png' for p in pathlist ]
        self.targets = torch.LongTensor(targets)
        self.size=size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        y = self.targets[idx]

        x = Image.open(img_path).resize(self.size, Image.BILINEAR) # all images are resized to (224,224)
        s = Image.open(mask_path).resize(self.size, Image.NEAREST) # all segmentation masks are resized to (224,224)

        # Random crop
        i, j, h, w = transforms.RandomCrop.get_params(x, output_size=(128, 128))
        x = TF.crop(x, i, j, h, w)
        s = TF.crop(s, i, j, h, w)

        # Random horizontal flipping
        if np.random.random() > 0.5:
          x = TF.hflip(x)
          s = TF.hflip(s)

        # Random vertical flipping
        if np.random.random() > 0.5:
            x = TF.vflip(x)
            s = TF.vflip(s)

        # Random rotation
        angle=np.random.randint(-90, 90)
        x=TF.rotate(x, angle, InterpolationMode.BILINEAR)
        s=TF.rotate(s,angle, InterpolationMode.NEAREST)

        # Multiply image and segmentation mask
        blank = x.point(lambda _: 0)
        c = Image.composite(x, blank, s)

        # Send to tensor
        x = TF.to_tensor(x)
        c = TF.to_tensor(c)

        # Normalize according to ImageNet statistics
        x = TF.normalize(x,(0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        c = TF.normalize(c,(0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

        return x, c, y




You have probably noticed that there are three outputs.

**Question**: Explain what they are and which data augmentations we are computing.

XXXXXXX

Then, we can load the data

In [ ]:
# to make the results reproducible
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

# Ensure that you are using GPU
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

batch_size=256 # adapted to the Google Colab GPU
num_epochs=1 # to be modified
learning_rate=0.05 # to be modified

train_dataset = ISICDataset(pathTrain, Ytrain)
val_dataset=ISICDataset(pathVal, Yval)
test_dataset=ISICDataset(pathTest, Ytest)

train_loader = DataLoader(train_dataset, num_workers=1, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, num_workers=1, batch_size=batch_size)
test_loader = DataLoader(test_dataset, num_workers=1, batch_size=batch_size)

The first part will be about fine-tuning the last layer of a network already pre-trained on Image-Net. Here it is the code:

In [ ]:
num_classes = 1 # since we use the BCEWithLogitsLoss, we only have one output (if you were using CrossEntropy Loss, num_classes would be 2)

# Load a pre-trained ResNet-18 model
resnet18 = models.resnet18(weights='IMAGENET1K_V1')
# Freeze all parameters of the model
for param in resnet18.parameters():
    param.requires_grad = False
# Change last layer, the fully connected layer (classifier)
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, num_classes)

Then, we can train this last layer on the training data. Find the best model in the validation set and evaluate the generalization performance on the test set. If you look for the best hyper-parameter, you should also use the validation set.

In [ ]:
from tqdm import tqdm

model=resnet18.float().cuda() # use float32 to save a bit of memory

# pos_weight is N_Train/sum(Ytrain) -> why in your opinion ?
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([5.347593582887701]).cuda())
auc = torchmetrics.AUROC(task='binary',num_classes=2, average = 'macro').cuda()
accuracy = torchmetrics.Accuracy(task='binary',num_classes = 2, average='micro').cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
Tensor = torch.cuda.FloatTensor # use float32 to save a bit of memory

# Training
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, composite, labels in tqdm(train_loader):
        optimizer.zero_grad()
        #outputs = model(images.float().cuda())
        outputs = model(composite.float().cuda())
        loss = criterion(outputs.squeeze(), labels.float().cuda())
        loss.backward()
        optimizer.step()
        #train_loss += loss.item() * images.size(0)
        train_loss += loss.item() * composite.size(0)
    train_loss = train_loss / len(train_dataset)

    all_preds=[]
    all_labels=[]
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, composite, labels in tqdm(val_loader):
            #outputs = model(images.float().cuda())
            outputs = model(composite.float().cuda())
            loss = criterion(outputs.squeeze(), labels.float().cuda())
            #val_loss += loss.item() * images.size(0)
            val_loss += loss.item() * composite.size(0)
            preds=torch.sigmoid(outputs).squeeze()
            auc.update(preds, labels.cuda())
            accuracy.update(preds, labels.cuda())
    val_loss = val_loss / len(val_dataset)
    val_auc = auc.compute()
    val_acc = accuracy.compute()
    print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}, Val AUC: {:.4f}, Val Acc: {:.4f}'.format(epoch+1, num_epochs, train_loss, val_loss, val_auc, val_acc))



In [ ]:
### Testing
model.eval()
test_loss=0
with torch.no_grad():
    for images, composite, labels in tqdm(test_loader):
        outputs = model(images.float().cuda())
        loss = criterion(outputs.squeeze(), labels.float().cuda())
        test_loss += loss.item()
        preds=torch.sigmoid(outputs).squeeze()
        auc.update(preds, labels.cuda())
        accuracy.update(preds, labels.cuda())
test_loss = test_loss / len(test_dataset)
test_auc = auc.compute()
test_acc = accuracy.compute()
print('\nTest Loss: {:.4f}, Test AUC: {:.4f}, Test Acc: {:.4f}'.format(test_loss, test_auc, test_acc))

Now, it's time to retrain the entire network (previously pre-trained on Imagenet)


In [ ]:
XXXXXXXXXX

And eventually train from scratch the network, namely the weights should be randomly initiliazed.

In [ ]:
XXXXXXXXXX

**Question**: Train at least 3 different networks using the three strategies. Use the validation set to compare the performance of the three models and evaluate the best-performing one on the test set. Which is the best strategy ? Are you satisfied ?

XXXXXXXX

**Question**: You can use as input to your model either the full image or the masked image. Compare the two strategies by testing *in both cases* only on the full images (assume that you do not have the segmentation masks at test time). In this way, the comparison will be more fair. What's better ?

XXXXXX

**Question**: Plot on a graph the time you have spent (x-axis) and the maximum AUC (y-axis) for all strategies you tried: feature engineering and DL fie-tuning or training. What's your conclusion ?

XXXXXXX